# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [73]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from resources import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [74]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data.head()

,city,lat,lon,temp,humidity,cloudiness,wind speed
0,tautira,-17.73,-149.15,84.20,54,20,2.24
1,busselton,-33.65,115.33,58.32,47,11,11.70
2,kapaa,22.08,-159.32,82.58,66,75,12.75
3,riohacha,11.54,-72.91,82.51,78,3,11.92
4,kathu,7.92,98.33,81.05,94,75,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [75]:
#API module Day 3, Activity 10

#configure gmaps with API key
gmaps.configure(api_key=gkey)

In [76]:
#store lat and lng into locations as floats
locations = weather_data[["lat","lon"]].astype(float)

#store humidity as float
#https://datatofish.com/convert-string-to-float-dataframe/
weather_data["humidity"] = weather_data["humidity"].astype(float)

In [77]:
fig = gmaps.figure(center = (20,0), zoom_level = 2.2)

heat_layer = gmaps.heatmap_layer(locations, weights=weather_data['humidity'], 
                                 dissipating=False, max_intensity=100, point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [106]:
#narrow cities down to those with ideal weather conditions
ideal_weather_df = weather_data.loc[(weather_data["temp"] >= 75) & (weather_data["temp"] <=85) & 
                                    (weather_data["humidity"] < 70) & (weather_data["wind speed"] < 20)]

#drop cities with missing data
ideal_weather_df.dropna

#view df
ideal_weather_df.head()

,city,lat,lon,temp,humidity,cloudiness,wind speed
0,tautira,-17.73,-149.15,84.20,54.0,20,2.24
2,kapaa,22.08,-159.32,82.58,66.0,75,12.75
27,buraydah,26.33,43.98,80.60,32.0,35,4.70
55,hilo,19.73,-155.09,79.21,51.0,1,10.29
61,feicheng,35.26,117.97,83.41,65.0,0,6.49


In [107]:
#count remaining cities
len(ideal_weather_df)

42

In [160]:
#reset index
# ideal_weather_df = ideal_weather_df.reset_index()
# ideal_weather_df.head()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [162]:
# My original attempt
# hotel_df = pd.DataFrame()
# hotel_df['Hotel Name'] = ""
# hotel_df["lat"] = ""
# hotel_df["lon"] = ""
# hotel_df["City"] = ""
# hotel_df["Country"] = ""

#after Ask BCS assistance
hotel_df = ideal_weather_df[["city","lat","lon"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,city,lat,lon,Hotel Name
0,tautira,-17.73,-149.15,
2,kapaa,22.08,-159.32,
27,buraydah,26.33,43.98,
55,hilo,19.73,-155.09,
61,feicheng,35.26,117.97,
91,avarua,-21.21,-159.78,
100,trincomalee,8.57,81.23,
120,hervey bay,-25.30,152.85,
147,linhai,28.85,121.12,
167,namatanai,-3.67,152.43,


In [163]:
#set parameters for hotel search
params = {
    "radius" : 5000,
    "types": "lodging",
    "key": gkey
}

#capture data
for index,row in ideal_weather_df.iterrows():
    
    #geocoordinates
    lat = row["lat"]
    lon = row["lon"]
    
    params["location"] = f"{lat},{lon}"
    
    #base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #make request, print url and convert to json
    request = requests.get(base_url, params=params)
    response = request.json()

    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        
    except (KeyError,IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.


,city,lat,lon,Hotel Name
0,tautira,-17.73,-149.15,Pension Oaoa
2,kapaa,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
27,buraydah,26.33,43.98,Mövenpick Hotel Qassim
55,hilo,19.73,-155.09,Hilo Hawaiian Hotel
61,feicheng,35.26,117.97,Royal Garden Hotel
91,avarua,-21.21,-159.78,Paradise Inn
100,trincomalee,8.57,81.23,Welcombe Hotel
120,hervey bay,-25.30,152.85,Shelly Bay Resort
147,linhai,28.85,121.12,Linhai S&N International Hotel
167,namatanai,-3.67,152.43,Namatanai Hotel


In [164]:
len(hotel_df)

42

In [167]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [168]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

#I woud like to display the hotel name, city and country on the map but am out of time


Figure(layout=FigureLayout(height='420px'))